# FCS time-series dataset released by WFP 

The complete version of the data released by WFP regarding the Food Consumption Score (FCS).

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [1]:
from ipywidgets import interact, widgets, fixed
import plotly.graph_objects as go
import seaborn as sns
from scipy.interpolate import splrep, splev
import pandas as pd
import numpy as np
import ntpath
import glob

In [2]:
# Read the data released by wfp regarding the FCS indicator.
path = "./wfp_data/"
all_files = glob.glob(path + "*.csv")

dfs = []

for filename in all_files:
    df = pd.read_csv(filename)
    df["Country"] = ntpath.basename(filename).split(".")[0]
    dfs.append(df)

In [3]:
print("The data released by wfp:")
df = pd.concat(dfs, axis = 0, ignore_index = True)
df.head()

The data released by wfp:


,SvyDate,Date,Dmgrph,DmgrphCode,Mean_crrnt,Pop,PopNmbr,Country
0,2019-7-15_2019-9-8,2019-09-08,BOUCLE-DU-MOUHOUN,900712.0,30.921705,1976217,611079,Burkina Faso
1,2019-7-15_2019-9-8,2019-09-08,CASCADES,900713.0,22.732405,822445,186961,Burkina Faso
2,2019-7-15_2019-9-8,2019-09-08,CENTRE,900714.0,22.093784,2854356,630635,Burkina Faso
3,2019-7-15_2019-9-8,2019-09-08,CENTRE-EST,900715.0,44.186047,2854356,1261227,Burkina Faso
4,2019-7-15_2019-9-8,2019-09-08,CENTRE-NORD,900716.0,61.948843,1687858,1045608,Burkina Faso


### Brief items description

- *SvyDate*: reference period of the data collection.
- *Date*: reference date (i.e. the end of the reference period). 
- *Dmgrph*: administrative area name.
- *DmgrphCode*: a code for identyfing the adminstrata.
- *Mean_crrnt*: % of people with poor and borderline FCS.
- *Pop*: area population size.
- *PopNmbr*: number of people with poor and borderline FCS (i.e. Mean_crrnt * Pop).

In [4]:
# Check if the dataframe contains NaN values.
print("Check if the dataframe contains NaN values:")
df.isnull().sum()

Check if the dataframe contains NaN values:


SvyDate          0
Date             0
Dmgrph           0
DmgrphCode    1620
Mean_crrnt       0
Pop              0
PopNmbr          0
Country          0
dtype: int64

In [5]:
# Let's delete the item 'DmgrphCode'.
df.drop(["DmgrphCode"], axis = 1, inplace = True) 

In [6]:
# Rename some columns.
df.rename(columns = {"Date": "Datetime", "Dmgrph": "AdminStrata", "Mean_crrnt": "Metric"}, inplace = True)

In [7]:
df.head()

,SvyDate,Datetime,AdminStrata,Metric,Pop,PopNmbr,Country
0,2019-7-15_2019-9-8,2019-09-08,BOUCLE-DU-MOUHOUN,30.921705,1976217,611079,Burkina Faso
1,2019-7-15_2019-9-8,2019-09-08,CASCADES,22.732405,822445,186961,Burkina Faso
2,2019-7-15_2019-9-8,2019-09-08,CENTRE,22.093784,2854356,630635,Burkina Faso
3,2019-7-15_2019-9-8,2019-09-08,CENTRE-EST,44.186047,2854356,1261227,Burkina Faso
4,2019-7-15_2019-9-8,2019-09-08,CENTRE-NORD,61.948843,1687858,1045608,Burkina Faso


### Country item

In [8]:
print("The countries are:", ", ".join(df.Country.unique()))

The countries are: Burkina Faso, Nigeria, Syria, Yemen


### AdminStrata item

In [9]:
AdminStratas = df.groupby("Country")["AdminStrata"].unique()

- **Yemen**: the administrative division of Yemen is divided into two main divisions (governorates and districts). There are 22 governorates, including the capital Sana'a (Amanat Al Asimah) and Socotra Archipelago. The FCS dataframe has the values of all the 22 governorates.

In [10]:
print(AdminStratas["Yemen"].shape)
AdminStratas["Yemen"]

(22,)


array(['Abyan', 'Al Hudaydah', 'Al Jawf', 'Al Maharah', 'Al Mahwit',
       'Amanat Al Asimah', 'Amran', 'Dhamar', 'Hadramaut', 'Hajjah',
       'Ibb', 'Lahj', 'Marib', 'Raymah', "Sa'ada", "Sana'a", 'Shabwah',
       'Taizz', 'Aden', 'Al Bayda', "Al Dhale'e", 'Socotra'], dtype=object)

- **Nigeria**: Nigeria is divided into 36 states. It also includes *The Federal Capital Territory* that is not a state and it is under the direct control of the federal government. The AdminStrata items of the FCS dataframe cover 3 states (Adamawa, Borno and Yobe) and each of these state is diveded into three parts: North, South, Central.

In [11]:
print(AdminStratas["Nigeria"].shape)
AdminStratas["Nigeria"]

(9,)


array(['Adamawa Central', 'Adamawa North', 'Adamawa South',
       'Borno Central', 'Borno North', 'Borno South', 'Yobe East',
       'Yobe North', 'Yobe South'], dtype=object)

- **Syria**: the governatores of the Syria country are 14. The FCS dataframe has the values of 13 governorates (Idlib governatore not included).

In [12]:
print(AdminStratas["Syria"].shape)
AdminStratas["Syria"]

(13,)


array(['Tartous', 'Al-Hasakeh', 'Aleppo', 'As-Sweida', 'Damascus',
       "Dar'a", 'Hama', 'Homs', 'Lattakia', 'Rural Damascus', 'Ar-Raqqa',
       'Deir-ez-Zor', 'Quneitra'], dtype=object)

- **Burkina Faso**: Burkina Faso is divided into 13 administrative regions. The FCS dataframe has the values of all 13 regions.

In [13]:
print(AdminStratas["Burkina Faso"].shape)
AdminStratas["Burkina Faso"]

(13,)


array(['BOUCLE-DU-MOUHOUN', 'CASCADES', 'CENTRE', 'CENTRE-EST',
       'CENTRE-NORD', 'CENTRE-OUEST', 'CENTRE-SUD', 'EST',
       'HAUTS-BASSINS', 'NORD', 'PLATEAU-CENTRAL', 'SAHEL', 'SUD-OUEST'],
      dtype=object)

In [14]:
# The strings of AdminStrata key for Burkina Faso country are uppercase, I change them to obtain lowercase with first capital letter.
def to_lower(country, admin):
    if country == "Burkina Faso":
        admin = admin.lower().title()      
    return admin
    
df["AdminStrata"] = df[["Country", "AdminStrata"]].apply(lambda x: to_lower(*x), axis = 1)

In [15]:
# Check the min e max values of the Metric.
print("The min and max values of the Metric:")
print(df.Metric.min(), ",", df.Metric.max())

The min and max values of the Metric:
2.24791265253693 , 84.55928979074173


In [16]:
# Creation of an accurate datetime format (not consider the item 'SvyDate').
df.drop(["SvyDate"], axis = 1, inplace = True) 
df["Datetime"] = pd.to_datetime(df["Datetime"])
df.sort_values("Datetime", ascending = True, inplace = True) # Sort on datetime.
df = df.groupby(["Country", "AdminStrata"]).apply(lambda group: group.set_index("Datetime").resample("D").mean()).reset_index()
df.reset_index(drop = True, inplace = True)
df.head()

,Country,AdminStrata,Datetime,Metric,Pop,PopNmbr
0,Burkina Faso,Boucle-Du-Mouhoun,2019-09-08,30.921705,1976217.0,611079.0
1,Burkina Faso,Boucle-Du-Mouhoun,2019-09-09,31.103928,1976217.0,614681.0
2,Burkina Faso,Boucle-Du-Mouhoun,2019-09-10,32.030296,1976217.0,632988.0
3,Burkina Faso,Boucle-Du-Mouhoun,2019-09-11,32.167300,1976217.0,635695.0
4,Burkina Faso,Boucle-Du-Mouhoun,2019-09-12,31.770045,1976217.0,627845.0


In [17]:
# Create a dataframe with multi index column in order to have a summary dataframe of the time-series.
df.drop(labels = ["Pop", "PopNmbr"], axis = 1, inplace = True)
df = df.set_index(["Datetime", "Country", "AdminStrata"]).unstack(["Country", "AdminStrata"])
df.columns = df.columns.droplevel(0)
df.columns = pd.MultiIndex.from_tuples(list(map(lambda x: tuple(list(x) + ["FCS"]), df.columns)))
df.columns.rename("Country", level = 0, inplace = True)
df.columns.rename("AdminStrata", level = 1, inplace = True)
df.columns.rename("Indicator", level = 2, inplace = True)
freq = "D"
df.index.freq = freq
df.head()

Country          Burkina Faso                                         \
AdminStrata Boucle-Du-Mouhoun Cascades Centre Centre-Est Centre-Nord   
Indicator                 FCS      FCS    FCS        FCS         FCS   
Datetime                                                               
2018-07-02                NaN      NaN    NaN        NaN         NaN   
2018-07-03                NaN      NaN    NaN        NaN         NaN   
2018-07-04                NaN      NaN    NaN        NaN         NaN   
2018-07-05                NaN      NaN    NaN        NaN         NaN   
2018-07-06                NaN      NaN    NaN        NaN         NaN   

Country                                                     ...      Yemen  \
AdminStrata Centre-Ouest Centre-Sud Est Hauts-Bassins Nord  ...     Hajjah   
Indicator            FCS        FCS FCS           FCS  FCS  ...        FCS   
Datetime                                                    ...              
2018-07-02           NaN        NaN NaN           NaN  NaN  ...  28.361345   
2018-07-03           NaN        NaN NaN           NaN  NaN  ...  28.805448   
2018-07-04           NaN        NaN NaN           NaN  NaN  ...  30.993706   
2018-07-05           NaN        NaN NaN           NaN  NaN  ...  32.905902   
2018-07-06           NaN        NaN NaN           NaN  NaN  ...  32.598181   

Country                                                                        \
AdminStrata        Ibb       Lahj      Marib     Raymah     Sa'ada     Sana'a   
Indicator          FCS        FCS        FCS        FCS        FCS        FCS   
Datetime                                                                        
2018-07-02   25.036668  31.762436  32.596233  54.121774  17.676022  26.917713   
2018-07-03   27.102390  32.049499  33.333333  56.722689  17.134560  25.783476   
2018-07-04   29.844066  33.106267  36.883683  56.306306  17.454545  25.436047   
2018-07-05   29.025363  32.938828  36.897633  56.231003  17.846350  27.421759   
2018-07-06   30.692766  33.837934  36.669568  57.324841  18.463057  26.547231   

Country                                       
AdminStrata    Shabwah    Socotra      Taizz  
Indicator          FCS        FCS        FCS  
Datetime                                      
2018-07-02   16.752289  23.822825  26.563365  
2018-07-03   18.257453  23.699881  29.124005  
2018-07-04   19.083081  23.103187  29.427973  
2018-07-05   20.349533  21.596419  30.456026  
2018-07-06   21.107524  20.129084  31.012517  

[5 rows x 57 columns]

In [18]:
def plot(country, graph, df, style, save, name):
    group = df[country]
    # Adjust time-series group.
    group.columns = group.columns.droplevel(1)
    first_idx = group.first_valid_index()
    last_idx = group.last_valid_index()
    group = group.loc[first_idx:last_idx]
    
    if graph == "Time-series":
        # Create figure.
        # Set default trace colors with colorway.
        colorway = sns.color_palette("hls", 8).as_hex()
        layout = go.Layout(colorway = colorway)

        fig = go.Figure(layout = layout)

        for col in group.columns:
            fig.add_trace(go.Scatter(x = group.index, y = group[col], name = col, mode = style, line = dict(width = 1.5)))

        # Edit the layout.
        fig.update_layout(title = dict(text = country, y = 0.9, x = 0.5), 
                          yaxis_title = dict(text = "% of people with poor and borderline FCS", font = dict(size = 10)))
        # Add legend title.
        fig.update_layout(annotations = [dict(x = 1.12, y = 1.1, align = "right", valign = "top", text = "AdminStrata", 
                                              showarrow = False, xref = "paper", yref = "paper", xanchor = "center", 
                                              yanchor = "top")])
        # Add range slider.
        fig.update_layout(xaxis = dict(title = "Datetime", rangeselector = dict(), rangeslider = dict(visible = True), 
                                       type = "date"))

        fig.show()
        
    if graph == "Missing values":    
        # Visualization of the missing values of the current time-series.
        miss = group.notnull().astype("int")
        fig1 = go.Figure(data = go.Heatmap(z = miss, zmin = 0, zmax = 1, x = miss.columns, y = miss.index, xgap = 3, 
                                           colorscale = [[0, "white"], [1, "black"]], showscale = False, 
                                           hoverinfo = "x+y"))
        # Edit the layout.
        fig1.update_layout(title = dict(text = country, y = 0.9, x = 0.5), coloraxis_showscale = False)
        fig1.update_layout(xaxis_title = "Adminstrata", yaxis_title = "Datetime")
        fig1.show()
    
    if save:
        # Reading the column index to indicate the corresonding indicator.
        group.columns = pd.MultiIndex.from_product([group.columns, ["FCS"]])
        # Save group.
        group.to_csv("./time-series/" + name + country + ".csv", index_label = False)

In [19]:
w1 = widgets.ToggleButtons(options = df.columns.get_level_values(0).unique(), description = "Country:", disabled = False)
w2 = widgets.RadioButtons(options = ["Time-series", "Missing values"], description = " ", disabled = False)
p = interact(plot, country = w1, graph = w2, df = fixed(df), style = fixed("lines"), save = fixed(False), name = fixed("wfp_fcs_"))

interactive(children=(ToggleButtons(description='Country:', options=('Burkina Faso', 'Nigeria', 'Syria', 'Yeme…

ATTENTION: the adminstratas 'Hadramaut' and 'Socotra' of the Yemen country have the same time-series of the FCS indicator! I delete the time-series of the 'Socotra' adminstrata. 

In [20]:
df = df.drop("Socotra", axis = 1, level = 1)

## Adjusting the time-series

In [21]:
# I get the time-series for each country without nan values in the middle and with equal start and end between adminstrata of the same country.
def interpolation(group):   
    group.columns = group.columns.droplevel()
    first_idx = group.first_valid_index()
    last_idx = group.last_valid_index()
    group = group.loc[first_idx:last_idx]
    group = group.interpolate(method = "linear", limit = 7)
    # Delete time-series that still have some NaN values.
    group.dropna(inplace = True, axis = 1)
    return group

df_interpolate = df.groupby(axis = 1, level = 0).apply(interpolation)
df_interpolate.head()

Country          Burkina Faso                                         \
AdminStrata Boucle-Du-Mouhoun Cascades Centre Centre-Est Centre-Nord   
Indicator                 FCS      FCS    FCS        FCS         FCS   
Datetime                                                               
2018-07-02                NaN      NaN    NaN        NaN         NaN   
2018-07-03                NaN      NaN    NaN        NaN         NaN   
2018-07-04                NaN      NaN    NaN        NaN         NaN   
2018-07-05                NaN      NaN    NaN        NaN         NaN   
2018-07-06                NaN      NaN    NaN        NaN         NaN   

Country                                                     ...      Yemen  \
AdminStrata Centre-Ouest Centre-Sud Est Hauts-Bassins Nord  ...  Hadramaut   
Indicator            FCS        FCS FCS           FCS  FCS  ...        FCS   
Datetime                                                    ...              
2018-07-02           NaN        NaN NaN           NaN  NaN  ...  23.822825   
2018-07-03           NaN        NaN NaN           NaN  NaN  ...  23.699881   
2018-07-04           NaN        NaN NaN           NaN  NaN  ...  23.103187   
2018-07-05           NaN        NaN NaN           NaN  NaN  ...  21.596419   
2018-07-06           NaN        NaN NaN           NaN  NaN  ...  20.129084   

Country                                                                        \
AdminStrata     Hajjah        Ibb       Lahj      Marib     Raymah     Sa'ada   
Indicator          FCS        FCS        FCS        FCS        FCS        FCS   
Datetime                                                                        
2018-07-02   28.361345  25.036668  31.762436  32.596233  54.121774  17.676022   
2018-07-03   28.805448  27.102390  32.049499  33.333333  56.722689  17.134560   
2018-07-04   30.993706  29.844066  33.106267  36.883683  56.306306  17.454545   
2018-07-05   32.905902  29.025363  32.938828  36.897633  56.231003  17.846350   
2018-07-06   32.598181  30.692766  33.837934  36.669568  57.324841  18.463057   

Country                                       
AdminStrata     Sana'a    Shabwah      Taizz  
Indicator          FCS        FCS        FCS  
Datetime                                      
2018-07-02   26.917713  16.752289  26.563365  
2018-07-03   25.783476  18.257453  29.124005  
2018-07-04   25.436047  19.083081  29.427973  
2018-07-05   27.421759  20.349533  30.456026  
2018-07-06   26.547231  21.107524  31.012517  

[5 rows x 53 columns]

In [22]:
w1 = widgets.ToggleButtons(options = df_interpolate.columns.get_level_values(0).unique(), description = "Country:", disabled = False)
w2 = widgets.RadioButtons(options = ["Time-series", "Missing values"], description = " ", disabled = False)
p = interact(plot, country = w1, graph = w2, df = fixed(df_interpolate), style = fixed("lines"), save = fixed(True), name = fixed("wfp_fcs_"))

interactive(children=(ToggleButtons(description='Country:', options=('Burkina Faso', 'Nigeria', 'Syria', 'Yeme…

## Fit of the time-series (smooth data)

In [23]:
def fit(group):   
    group.columns = group.columns.droplevel()
    # Delete time-series that still have some NaN values.
    group.dropna(inplace = True, axis = 0)
    
    def smooth(serie):
        bspl = splrep(np.arange(0, len(serie)), serie.values, s = 500)
        bspl_y = splev(np.arange(0, len(serie)), bspl)
        return pd.Series(bspl_y, index = serie.index, name = serie.name)
    
    group_fit = group.apply(smooth)

    return group_fit

df_fit = df_interpolate.groupby(axis = 1, level = 0).apply(fit)
df_fit.dropna(axis = 0, how = "all", inplace = True)
df_fit.head()

Country          Burkina Faso                                         \
AdminStrata Boucle-Du-Mouhoun Cascades Centre Centre-Est Centre-Nord   
Indicator                 FCS      FCS    FCS        FCS         FCS   
Datetime                                                               
2018-07-02                NaN      NaN    NaN        NaN         NaN   
2018-07-03                NaN      NaN    NaN        NaN         NaN   
2018-07-04                NaN      NaN    NaN        NaN         NaN   
2018-07-05                NaN      NaN    NaN        NaN         NaN   
2018-07-06                NaN      NaN    NaN        NaN         NaN   

Country                                                     ...      Yemen  \
AdminStrata Centre-Ouest Centre-Sud Est Hauts-Bassins Nord  ...  Hadramaut   
Indicator            FCS        FCS FCS           FCS  FCS  ...        FCS   
Datetime                                                    ...              
2018-07-02           NaN        NaN NaN           NaN  NaN  ...  24.399919   
2018-07-03           NaN        NaN NaN           NaN  NaN  ...  23.258033   
2018-07-04           NaN        NaN NaN           NaN  NaN  ...  22.328133   
2018-07-05           NaN        NaN NaN           NaN  NaN  ...  21.592889   
2018-07-06           NaN        NaN NaN           NaN  NaN  ...  21.034973   

Country                                                                        \
AdminStrata     Hajjah        Ibb       Lahj      Marib     Raymah     Sa'ada   
Indicator          FCS        FCS        FCS        FCS        FCS        FCS   
Datetime                                                                        
2018-07-02   28.007419  24.863115  32.017741  33.008420  54.694139  16.948502   
2018-07-03   29.833781  27.222605  32.325814  34.231691  55.408640  17.488985   
2018-07-04   31.083657  29.025828  32.631356  35.330495  56.373386  17.986721   
2018-07-05   31.843099  30.351675  32.932649  36.315678  57.384752  18.443266   
2018-07-06   32.198159  31.279039  33.227976  37.198090  58.239116  18.860173   

Country                                       
AdminStrata     Sana'a    Shabwah      Taizz  
Indicator          FCS        FCS        FCS  
Datetime                                      
2018-07-02   25.795141  16.834657  28.095998  
2018-07-03   26.206560  18.236068  28.649019  
2018-07-04   26.609465  19.328230  29.151062  
2018-07-05   27.001042  20.158842  29.604903  
2018-07-06   27.378479  20.775606  30.013316  

[5 rows x 53 columns]

In [24]:
w1 = widgets.ToggleButtons(options = df_fit.columns.get_level_values(0).unique(), description = "Country:", disabled = False)
w2 = widgets.RadioButtons(options = ["Time-series", "Missing values"], description = " ", disabled = False)
p = interact(plot, country = w1, graph = w2, df = fixed(df_fit), style = fixed("lines"), save = fixed(True), name = fixed("wfp_fcs_smooth_"))

interactive(children=(ToggleButtons(description='Country:', options=('Burkina Faso', 'Nigeria', 'Syria', 'Yeme…

### Example fit operation

In [26]:
Country = "Yemen"

In [27]:
subdf_fit = df_fit[Country]
subdf_fit.columns = subdf_fit.columns.droplevel(1)

In [28]:
subdf_interpolate = df_interpolate[Country]
subdf_interpolate.columns = subdf_interpolate.columns.droplevel(1)

In [29]:
# Create figure.
fig = go.Figure()

colorway = sns.color_palette("hls", 8).as_hex()

for i, col in enumerate(subdf_fit.columns):
    fig.add_trace(go.Scatter(x = subdf_fit.index, y = subdf_fit[col], name = col, line = dict(width = 1.5, 
                                                                                              color = colorway[i % len(colorway)]), 
                             legendgroup = col))
    fig.add_trace(go.Scatter(x = subdf_interpolate.index, y = subdf_interpolate[col], name = col, line = dict(width = 1., 
                                                                                                              color = colorway[i % len(colorway)]), 
                             showlegend = False, legendgroup = col))

# Edit the layout.
fig.update_layout(title = dict(text = Country, y = 0.9, x = 0.5), 
                  yaxis_title = dict(text = "% of people with poor and borderline FCS", font = dict(size = 10)))
# Add legend title.
fig.update_layout(annotations = [dict(x = 1.12, y = 1.1, align = "right", valign = "top", text = "AdminStrata", 
                                      showarrow = False, xref = "paper", yref = "paper", xanchor = "center", 
                                      yanchor = "top")])
# Add range slider.
fig.update_layout(xaxis = dict(title = "Datetime", rangeselector = dict(), rangeslider = dict(visible = True), 
                               type = "date"))

fig.show()

## Resampling datetime

In [30]:
mask = df.index.map(lambda x: x.day) == 1
df_resample = df_interpolate[mask]
# Resampling according to the survey date monthly.
df_resample.index = df_resample.index - pd.offsets.MonthEnd() - pd.offsets.MonthBegin() + pd.offsets.MonthEnd()
df_resample.head()

Country          Burkina Faso                                         \
AdminStrata Boucle-Du-Mouhoun Cascades Centre Centre-Est Centre-Nord   
Indicator                 FCS      FCS    FCS        FCS         FCS   
Datetime                                                               
2018-07-31                NaN      NaN    NaN        NaN         NaN   
2018-08-31                NaN      NaN    NaN        NaN         NaN   
2018-09-30                NaN      NaN    NaN        NaN         NaN   
2018-10-31                NaN      NaN    NaN        NaN         NaN   
2018-11-30                NaN      NaN    NaN        NaN         NaN   

Country                                                     ...      Yemen  \
AdminStrata Centre-Ouest Centre-Sud Est Hauts-Bassins Nord  ...  Hadramaut   
Indicator            FCS        FCS FCS           FCS  FCS  ...        FCS   
Datetime                                                    ...              
2018-07-31           NaN        NaN NaN           NaN  NaN  ...  14.190981   
2018-08-31           NaN        NaN NaN           NaN  NaN  ...  29.690346   
2018-09-30           NaN        NaN NaN           NaN  NaN  ...  55.187398   
2018-10-31           NaN        NaN NaN           NaN  NaN  ...  46.774194   
2018-11-30           NaN        NaN NaN           NaN  NaN  ...  23.325499   

Country                                                                        \
AdminStrata     Hajjah        Ibb       Lahj      Marib     Raymah     Sa'ada   
Indicator          FCS        FCS        FCS        FCS        FCS        FCS   
Datetime                                                                        
2018-07-31   34.535960  35.833333  35.454545  51.114874  54.224884  28.857253   
2018-08-31   28.928489  37.259657  48.269869  40.407938  47.733961  24.047744   
2018-09-30   36.924202  38.393188  46.142106  52.647555  58.202654  32.363977   
2018-10-31   44.526198  50.569260  54.809437  56.738132  64.000000  34.025354   
2018-11-30   40.631164  45.039284  46.892546  55.200613  50.889328  23.596838   

Country                                       
AdminStrata     Sana'a    Shabwah      Taizz  
Indicator          FCS        FCS        FCS  
Datetime                                      
2018-07-31   26.683584  27.110468  32.697150  
2018-08-31   29.478114  32.847257  36.374931  
2018-09-30   32.346201  42.878164  38.884275  
2018-10-31   32.532458  32.056137  51.048730  
2018-11-30   37.650000  39.366213  49.809614  

[5 rows x 53 columns]

In [31]:
w1 = widgets.ToggleButtons(options = df_resample.columns.get_level_values(0).unique(), description = "Country:", disabled = False)
w2 = widgets.RadioButtons(options = ["Time-series", "Missing values"], description = " ", disabled = False)
p = interact(plot, country = w1, graph = w2, df = fixed(df_resample), style = fixed("lines+markers"), save = fixed(False), name = fixed("wfp_fcs_resample_"))

interactive(children=(ToggleButtons(description='Country:', options=('Burkina Faso', 'Nigeria', 'Syria', 'Yeme…